In [66]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Conv2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Accuracy
from tensorflow.keras.callbacks import EarlyStopping

In [67]:
print("Num GPUs Avaible: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Avaible:  1


In [68]:
# menentukan direktori dari dataset training dan validation yang digunakan
train_dir = 'dataset/train'
valid_dir = 'dataset/valid'
test_dir = 'dataset/test'

In [79]:
# menentukan ukuran gambar pada proses training
# artinya setiap gambar pada dataset akan diubah ukurannya menjadi ke sebuah ukuran
# yang telah ditentukan
img_width, img_height, channel = (224, 224, 3)

In [70]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 7274 images belonging to 5 classes.


In [86]:
train_data.class_indices

{'bakso': 0, 'gado': 1, 'gudeg': 2, 'rendang': 3, 'sate': 4}

In [71]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_width, img_height),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 913 images belonging to 5 classes.


In [72]:
valid_datagen = ImageDataGenerator(
    rescale = 1./255
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_width),
    batch_size=16,
    class_mode='categorical'
)

Found 902 images belonging to 5 classes.


In [80]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import MobileNet

# base_model = VGG19(
#     input_shape=(img_width, img_height, channel),
#     include_top=False,
#     weights='imagenet'
# )
base_model = MobileNet(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights='imagenet'
)

for layer in base_model.layers:
    layer.trainable = False

In [81]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x)
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [82]:
from tensorflow.keras.metrics import Recall, Precision

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics= [Recall(), Precision(), "accuracy"]
)

In [83]:
model_callback = EarlyStopping(
    monitor = 'loss',
    verbose = 1,
    mode = 'min',
    patience= 3
)

history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=30,
    steps_per_epoch=50,
    callbacks = [model_callback],
    verbose=1
)

Epoch 1/30
50/50 [==============================] - 11s 184ms/step - loss: 0.8492 - recall_6: 0.5387 - precision_6: 0.8163 - accuracy: 0.6725 - val_loss: 0.4390 - val_recall_6: 0.8060 - val_precision_6: 0.8844 - val_accuracy: 0.8503
Epoch 2/30
50/50 [==============================] - 9s 172ms/step - loss: 0.4056 - recall_6: 0.8375 - precision_6: 0.8785 - accuracy: 0.8615 - val_loss: 0.3877 - val_recall_6: 0.8271 - val_precision_6: 0.8818 - val_accuracy: 0.8548
Epoch 3/30
50/50 [==============================] - 8s 167ms/step - loss: 0.3370 - recall_6: 0.8662 - precision_6: 0.8988 - accuracy: 0.8800 - val_loss: 0.2788 - val_recall_6: 0.8792 - val_precision_6: 0.9053 - val_accuracy: 0.8902
Epoch 4/30
50/50 [==============================] - 8s 166ms/step - loss: 0.3064 - recall_6: 0.8813 - precision_6: 0.9120 - accuracy: 0.8950 - val_loss: 0.2714 - val_recall_6: 0.8925 - val_precision_6: 0.9158 - val_accuracy: 0.9080
Epoch 5/30
50/50 [==============================] - 8s 170ms/step - los

In [58]:
model.evaluate(test_data)

58/58 [==============================] - 6s 98ms/step - loss: 0.4875 - recall_5: 0.7919 - precision_5: 0.8597 - accuracy: 0.8182


[0.4874841272830963,
 0.7918948531150818,
 0.8596908450126648,
 0.8181818127632141]

In [84]:
model.save('model_mobilenet_acc81.h5')

In [59]:
base_model.trainable = True

print('Number of layers : ', len(base_model.layers))

fine_tune_at = 15

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

Number of layers :  22


In [60]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 220, 220, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 220, 220, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 220, 220, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 110, 110, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 110, 110, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 110, 110, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 55, 55, 128)       0   

In [63]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.01),
    metrics= ["accuracy"]
)

In [64]:
model_callback = EarlyStopping(
    monitor = 'loss',
    verbose = 1,
    mode = 'min',
    patience= 3
)

history_fine = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=40,
    initial_epoch=history.epoch[-1],
    steps_per_epoch=50,
    callbacks = [model_callback],
    verbose=1
)

Epoch 14/40
50/50 [==============================] - 13s 242ms/step - loss: 49.3883 - accuracy: 0.2500 - val_loss: 1.5494 - val_accuracy: 0.2373
Epoch 15/40
50/50 [==============================] - 13s 267ms/step - loss: 16.2941 - accuracy: 0.2292 - val_loss: 1.5296 - val_accuracy: 0.2417
Epoch 16/40
50/50 [==============================] - 12s 244ms/step - loss: 1.5468 - accuracy: 0.2305 - val_loss: 1.5228 - val_accuracy: 0.2417
Epoch 17/40
50/50 [==============================] - 14s 279ms/step - loss: 1.5081 - accuracy: 0.2350 - val_loss: 1.5204 - val_accuracy: 0.2317
Epoch 18/40
50/50 [==============================] - 12s 246ms/step - loss: 1.5289 - accuracy: 0.2475 - val_loss: 1.5215 - val_accuracy: 0.2317
Epoch 19/40
50/50 [==============================] - 14s 283ms/step - loss: 1.5563 - accuracy: 0.2087 - val_loss: 1.5219 - val_accuracy: 0.2417
Epoch 20/40
50/50 [==============================] - 12s 246ms/step - loss: 1.5365 - accuracy: 0.2300 - val_loss: 1.5232 - val_accurac